<a href="https://colab.research.google.com/github/devdaveddev/DebateBOT/blob/main/refcodedb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet
from transformers import pipeline, BertForSequenceClassification, BertTokenizer, T5ForConditionalGeneration, T5Tokenizer
from textblob import TextBlob
import torch
import random
import numpy as np
import os
import tensorflow as tf
from bs4 import BeautifulSoup

# Initialize NLP models
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def fetch_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    return fetch_data(url).get("extract", "No information found.")

def fetch_stackexchange_data(topic):
    url = f"https://api.stackexchange.com/2.3/search?order=desc&sort=relevance&intitle={topic}&site=history"
    response = fetch_data(url)
    return [item["title"] for item in response.get("items", [])[:5]]

def fetch_reddit_data(topic):
    url = f"https://www.reddit.com/r/debaterelated/search.json?q={topic}&restrict_sr=1"
    headers = {'User-Agent': 'debate-bot'}
    response = fetch_data(url, headers)
    return [post["data"]["title"] for post in response.get("data", {}).get("children", [])[:5]]

def fetch_philosopher_data():
    philosophers = ["Socrates", "Plato", "Nietzsche", "Franz Kafka", "Neil deGrasse Tyson", "Alex O'Connor"]
    return {philosopher: fetch_wikipedia_data(philosopher) for philosopher in philosophers}

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score

# Transformer for Philosophical Style Adaptation
style_tokenizer = T5Tokenizer.from_pretrained("t5-small")
style_model = T5ForConditionalGeneration.from_pretrained("t5-small")

def adapt_style(text, philosopher):
    input_text = f"Rewrite as {philosopher}: {text}"
    inputs = style_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = style_model.generate(**inputs)
    return style_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Lexical Substitution for Philosophical Language Enhancement
def lexical_substitution(text):
    words = nltk.word_tokenize(text)
    new_text = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            new_text.append(synonyms[0].lemmas()[0].name())
        else:
            new_text.append(word)
    return ' '.join(new_text)

# ANN Model for Learning Debate Styles
class DebateANN(tf.keras.Model):
    def __init__(self):
        super(DebateANN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(5, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)

ann_model = DebateANN()
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TensorFlow Reinforcement Learning Model with LSTM
class DebateRLModel:
    def __init__(self):
        self.lstm_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(32, activation='relu', input_shape=(None, 5)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(5, activation='linear')
        ])
        self.lstm_model.compile(optimizer='adam', loss='mse')

        self.rl_model = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu', input_shape=(5,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(5, activation='linear')
        ])
        self.rl_model.compile(optimizer='adam', loss='mse')

        self.memory = []

    def train(self):
        if len(self.memory) > 100:
            batch = random.sample(self.memory, 100)
            X, y = zip(*batch)
            self.rl_model.fit(np.array(X), np.array(y), epochs=5, verbose=0)
            self.lstm_model.fit(np.array(X), np.array(y), epochs=5, verbose=0)

    def predict(self, X):
        rl_prediction = self.rl_model.predict(np.array([X]))[0]
        lstm_prediction = self.lstm_model.predict(np.array([X]))[0]
        ann_prediction = ann_model.predict(np.array([X]))[0]
        combined_prediction = (rl_prediction + lstm_prediction + ann_prediction) / 3
        return combined_prediction

    def remember(self, state, reward):
        self.memory.append((state, reward))
        if len(self.memory) > 1000:
            self.memory.pop(0)

rl_model = DebateRLModel()

def interactive_debate():
    print("Welcome to the AI Debate System!")
    mode = input("Choose mode: 1 for Philosophical Debate, 2 for Independent AI Response: ")
    if mode == "1":
        philosophers = ["Socrates", "Plato", "Aristotle", "Nietzsche", "Descartes"]
        print("Choose a philosopher:")
        for i, name in enumerate(philosophers):
            print(f"{i+1}. {name}")
        choice = int(input("Enter your choice: ")) - 1
        selected_philosopher = philosophers[choice]
        print(f"You are debating with {selected_philosopher}. Enter your argument:")
